<a href="https://colab.research.google.com/github/skyqi/LangChain/blob/main/langchain_ollama2%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_community
# !pip install --upgrade httpx httpx-sse PyJWT
#!pip install ollama
#!pip install -qU langchain-openai

In [11]:
#模型的第一次使用
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import Ollama

prompt_template = "What is a good name for a company that makes {product}?"

ollama_llm2 = Ollama(base_url='http://54.222.220.19:11435',model="llama3:8b")
llm_chain = LLMChain(
    llm = ollama_llm2,
    prompt = PromptTemplate.from_template(prompt_template)
)
print(llm_chain("colorful socks"))


{'product': 'colorful socks', 'text': 'What a fun question! Here are some ideas for a company name that captures the playful and vibrant spirit of colorful socks:\n\n1. HueHub: A play on the word "hue" (meaning color) and "hub," implying a central gathering place for all things colorful.\n2. SockSational: A whimsical name that combines "sock" with "sensational" to convey excitement and fun.\n3. ChromaCo.: "Chroma" means color, and the "-co." suffix suggests a company. This name has a modern, trendy feel.\n4. Footloose & Fancy Free: Inspired by the classic song, this name conveys a carefree spirit and a sense of playfulness.\n5. SoleMates: A pun on "soul mates," suggesting that your colorful socks will be the perfect match for customers\' feet (and personalities!).\n6. Pigment & Pride: This name highlights the vibrant colors and the pride that comes with wearing statement-making socks.\n7. ToeTally Colorful: Another playful name, this one emphasizing the fun and whimsy of your product.\

In [5]:
#这一个区的使用报错！！！
from langchain_openai import ChatOpenAI

LLM_ollama = ChatOpenAI(
    temperature=0.95,
    model="llama3:8b",
    openai_api_key="api-key",
    openai_api_base="http://54.222.220.19:11435"
)

In [10]:
from langchain.agents import initialize_agent, Tool
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

store = {}
# get_session_history 一直在保存。缺点容易引起输入token过大
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
#使用的ollama_llm2
with_message_history = RunnableWithMessageHistory(ollama_llm2,get_session_history)
config = {"configurable": {"session_id": "abc2"}}
response = with_message_history.invoke(
    [HumanMessage(content="今天是2024-6-17日周一，昨天是什么时间.请使用中文回复")],
    config=config,
)
response

'😊\n\n昨天是2024年6月16日星期日。'

In [23]:
# 格式化消息列表。这些“模板”由模板本身的列表组成。例如，构建和使用 ChatPromptTemplate 的常见方法如下：
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "今天2024-7-4，气温31度"),
    ("system", "请使用中文回复"),
    ("human", "{user_input_msg}")
])
# 定义用户输入
user_input = "今天出门出行要注意什么事项?"
# 使用模板生成聊天提示,仅是对模板进行修改
# chat_prompt = template.format(user_input_msg=user_input)
# # 仅是对模板进行修改，打印聊天提示
# print(chat_prompt)

llm_chain = LLMChain(
    llm = ollama_llm2,
    prompt = template
)

llm_chain.run(user_input_msg=user_input)

'🌞️今天是夏日的第一个星期四，你出门出行需要注意以下几点：\n\n1. 防晒：因为气温高达31°C，防晒非常重要。穿上遮阳服装，戴帽子，带太阳眼镜。\n2. hydrate：在夏季，人体容易脱水。确保喝足够的水，並帶一瓶水出来。\n3.避免炎热时段：如果可能，尽量避免出行在11点至15点之间，这个时段最热。\n4.携带防晒用品：带帽子、太阳眼镜和遮阳服装，以确保自己的皮肤保护。\n5.注意交通安全：夏季是高温期，路面温度较高，请注意行驶的路面和步行道。\n\n总之，出门出行时，需要注意防晒、hydrate和交通安全等问题。'

In [31]:
# ollama图片场景就识破别
from ollama import Client
client = Client(host='http://54.222.220.19:11435/')
response = client.chat(model='llava-llama3:latest', messages=[
  {
    'role': 'user',
    'content': '用中文六个字以内描述图片内的场景',
    'images': ["..."]
  },
])

print(response)

{'model': 'llava-llama3:latest', 'created_at': '2024-07-04T14:43:43.945713811Z', 'message': {'role': 'assistant', 'content': '该图片描绘了一个宁静的场景，有三个鸟在黄色天空中的两只鸟之间飞行。它们正朝着一座岛前进，远处可以看到陆地上的船只和码头。此外，还有一些草、木桩以及水域内的岩石。整个画面给人一种平静而美丽的感觉。'}, 'done_reason': 'stop', 'done': True, 'total_duration': 3265906974, 'load_duration': 5737733, 'prompt_eval_count': 1, 'prompt_eval_duration': 775820000, 'eval_count': 83, 'eval_duration': 2330278000}
